In [1]:
import pennylane as qml
import numpy as np

from qiskit.quantum_info import Operator

from src.CPQAOA import CP_QAOA
from src.QAOA import QAOA
from src.Chain import Chain
from src.Tools import (portfolio_metrics, 
                       min_cost_partition, 
                       get_qubo, 
                       normalized_cost, 
                       qubo_limits, 
                       check_qubo)
from src.Tools import get_qiskit_H

In [100]:
n_wires = 15
init_strat = np.array([1 if i%2 == 1 else 0 for i in range(n_wires)])
k = n_wires // 2
my_indices = [(i, i+1) for i in range(n_wires-1)]
n_layers = 4
alpha = 0.5
seed = 0
gamma_vals, beta_vals = np.random.uniform(-2*np.pi, 2*np.pi, n_layers), np.random.uniform(-2*np.pi, 2*np.pi, n_layers)
#TODO: Check out the 'lightning.gpu' plugin, which is a fast state-vector simulator offloading to the NVIDIA cuQuantum SDK for GPU accelerated circuit simulation. (not supported on windows...)
dev = qml.device('lightning.qubit', wires=n_wires)

In [101]:
 # Defining topology
my_chain = Chain(N_qubits=n_wires)
my_chain.set_initialization_strategy(strategy=init_strat)

# Deciding between grid and 1d chain topology
my_topology = my_chain

# Generating random problem instance 
expected_returns, covariances = portfolio_metrics(n=n_wires, seed=seed)

# Retrieving C_min, C_max and corresponding states for original portfolio problem
constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=n_wires,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

portfolio_subspace_max_cost, portfolio_subspace_min_cost, portfolio_subspace_min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
#full_space_max_cost = full_result['c_max']
portfolio_subspace_min_state_str = ''.join([str(_) for _ in portfolio_subspace_min_state])

# Generating QUBO corresponding to current problem instance
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances, 
                     alpha=alpha,
                     lmbda=lmbda+1e-8, # Adding small constant purposely
                     k=k)
QUBO_limits = qubo_limits(Q=Q,offset=offset)
qubo_min_cost, qubo_max_cost = QUBO_limits['c_min'], QUBO_limits['c_max']
qubo_min_state, qubo_max_state = QUBO_limits['min_state'], QUBO_limits['max_state']
check_qubo(QUBO_matrix=Q, QUBO_offset=offset, expected_returns=expected_returns, covariances=covariances, alpha=alpha, k=k)
qubo_min_state_str = ''.join([str(_) for _ in qubo_min_state])


if not portfolio_subspace_min_state_str == qubo_min_state_str:
    raise RuntimeError(f'portfolio_subspace_min_state_str: {portfolio_subspace_min_state_str}, qubo_min_state_str={qubo_min_state_str}'+f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not np.isclose(qubo_min_cost,portfolio_subspace_min_cost):
    raise RuntimeError(f'Min. cost of qubo is: {qubo_min_cost}, but min. cost of constrained portfolio is: {portfolio_subspace_min_cost}.')

if not qubo_max_cost >= portfolio_subspace_max_cost:
    raise RuntimeError(f'Max. cost of qubo: {qubo_max_cost}, max. cost of portfolio subspace: {portfolio_subspace_max_cost} (should be qubo max. >= constrained portfolio max)')

In [102]:
# unitary operator U_B with parameter beta
def U_B(beta, n_wires):
    for wire in range(n_wires):
        qml.RX(2 * beta, wires=wire)

# unitary operator U_C with parameter gamma
def U_C(gamma, indices):
    for edge in indices:
        wire1 = edge[0]
        wire2 = edge[1]
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])
        
@qml.qnode(dev)
def circuit(gammas, betas, indices, n_layers, n_wires):
    # apply Hadamards to get the n qubit |+> state
    for wire in range(n_wires):
        qml.Hadamard(wires=wire)
        
    # p instances of unitary operators
    for i in range(n_layers):
        U_C(gammas[i], indices)
        U_B(betas[i], n_wires)

    return qml.state()

def string_to_array(string_rep: str) -> np.ndarray:
    return np.array([int(bit) for bit in string_rep]).astype(np.float64)

def qubo_cost(state: np.ndarray, QUBO_matrix: np.ndarray) -> float:
    return np.dot(state, np.dot(QUBO_matrix, state))

def int_to_fixed_length_binary_array(number, num_bits):
    # Convert the number to binary and remove the '0b' prefix
    binary_str = bin(number)[2:]
    # Pad the binary string with zeros if necessary
    padded_binary_str = binary_str.zfill(num_bits)
    # Convert to a NumPy array of integers
    return padded_binary_str

def get_counts(state_vector: np.ndarray) -> dict:
    n_qubits = int(np.log2(len(state)))
    return {int_to_fixed_length_binary_array(number=idx, 
                                             num_bits=n_qubits): np.abs(state_vector[idx])**2 for idx in range(len(state_vector))}
def cost(Q, state_vector: np.ndarray) -> float:
    counts = get_counts(state_vector=state_vector)
    return np.mean([probability * qubo_cost(state=string_to_array(bitstring), QUBO_matrix=Q) for
                        bitstring, probability in counts.items()]) 

def cost_hc(state_vector: np.ndarray, H) -> float:
    return float(np.real(np.dot(state_vector.conj(), np.dot(H, state_vector))))

from scipy.sparse import csc_matrix, csr_matrix, kron, identity
def get_normal_H(Q: np.ndarray, flip: bool = False) -> np.ndarray:
    """ Generates H = \sum_ij q_ij(I_i-Z_i)/2(I_j-Z_j)/2 
    If flip is True, the same convention for indexing as in Qiskit is assumed
    where MSB is the right most qubit in a string.
    """
    I = identity(2, format='csr', dtype=np.float32)
    Z = csr_matrix(np.array([[1, 0], [0, -1]], dtype=np.float32))
    gate_map = {'Z': Z, 'I': I}
    def get_term(i: int):
        N = Q.shape[0]
        if i == N - 1:
            _mat_rep_ = gate_map['Z']
            _after_I_ = identity(2 ** (N - 1), format='csr')
            _mat_rep_ = kron(_mat_rep_, _after_I_)
        else:
            _before_I_ = identity(2 ** (N - i - 1), format='csr')
            _mat_rep_ = kron(_before_I_, gate_map['Z'])
            _after_I_ = identity(2 ** i, format='csr')
            _mat_rep_ = kron(_mat_rep_, _after_I_)
        return csr_matrix(_mat_rep_)

    def get_ij_term(i: int, j: int, Q: np.ndarray) -> csc_matrix:
        N = Q.shape[0]
        I_term = identity(2**Q.shape[0], format='csr', dtype=np.float32)
        if flip:
            Z_i_term = get_term(i=N-i-1)
            if i == j:
                Z_ij_term = I_term
                Z_j_term = Z_i_term
            else:
                Z_j_term = get_term(i=N-j-1)
                Z_ij_term = get_term(i=N-i-1) @ get_term(i=N-j-1)
            return Q[i, j] / 4.0 * (I_term - Z_i_term - Z_j_term + Z_ij_term)
        Z_i_term = get_term(i=i)
        if i == j:
            Z_ij_term = I_term
            Z_j_term = Z_i_term
        else:
            Z_j_term = get_term(i=j)
            Z_ij_term = get_term(i=i) @ get_term(i=j)
        return Q[i, j] / 4.0 * (I_term - Z_i_term - Z_j_term + Z_ij_term)
        
        
    
    H = csr_matrix(np.zeros(shape=(2**Q.shape[0], 2**Q.shape[1]), dtype=np.float32))
    for i in range(Q.shape[0]):
        for j in range(Q.shape[1]):
            H += get_ij_term(i, j, Q).astype(np.float32)
    return np.array(H.todense())

In [103]:
state = circuit(gamma_vals, beta_vals, my_indices, n_layers, n_wires)

In [109]:
c = cost(Q=Q, state_vector=state)

In [105]:
H = get_normal_H(Q,flip=False)

In [106]:
c2 = cost_hc(state,H)

In [107]:
H.shape, state.shape

((32768, 32768), (32768,))